# Generative Adversarial Networks

In [5]:
"""
Ultra-tiny GAN example that won't kill your kernel.
Trains on 32x32 flattened cat images using simple dense layers.
"""

import os, glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# -----------------------------
# Config
# -----------------------------
DATA_DIR = "./data/cats"
IMAGE_SIZE = 32
BATCH_SIZE = 8
NOISE_DIM = 32
STEPS = 100000  # total training iterations
SAVE_EVERY = 100
OUT_DIR = "./gan_tiny_out"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# Load data (small & safe)
# -----------------------------
def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, [IMAGE_SIZE, IMAGE_SIZE])
    img = tf.cast(img, tf.float32) / 127.5 - 1.0
    return img

def make_dataset(data_dir):
    files = [f for f in glob.glob(os.path.join(data_dir, "*"))
             if f.lower().endswith((".jpg",".jpeg",".png"))]
    if not files:
        raise RuntimeError(f"No cat images in {data_dir}")
    print(f"🐱 Found {len(files)} images.")
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.shuffle(len(files)).map(load_image).batch(BATCH_SIZE)
    return ds

# -----------------------------
# Very small models
# -----------------------------
def make_generator():
    return tf.keras.Sequential([
        layers.Input(shape=(NOISE_DIM,)),
        layers.Dense(128, activation="relu"),
        layers.Dense(IMAGE_SIZE * IMAGE_SIZE * 3, activation="tanh"),
        layers.Reshape((IMAGE_SIZE, IMAGE_SIZE, 3))
    ])

def make_discriminator():
    return tf.keras.Sequential([
        layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

# -----------------------------
# Save sample images
# -----------------------------
def save_images(generator, step):
    noise = tf.random.normal([9, NOISE_DIM])
    imgs = generator(noise, training=False)
    imgs = (imgs + 1) / 2.0
    fig, axes = plt.subplots(3, 3, figsize=(4,4))
    for i, ax in enumerate(axes.flatten()):
        ax.imshow(imgs[i])
        ax.axis("off")
    plt.tight_layout()
    out = os.path.join(OUT_DIR, f"cats_{step:04d}.png")
    plt.savefig(out)
    plt.close()
    print(f"💾 Saved {out}")

# -----------------------------
# Training loop
# -----------------------------
def train():
    ds = make_dataset(DATA_DIR)
    gen = make_generator()
    disc = make_discriminator()
    bce = tf.keras.losses.BinaryCrossentropy()
    g_opt = tf.keras.optimizers.Adam(1e-4)
    d_opt = tf.keras.optimizers.Adam(1e-4)

    for step, real in enumerate(ds.repeat(), start=1):
        noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

        # --- Train discriminator ---
        with tf.GradientTape() as tape:
            fake = gen(noise, training=True)
            real_pred = disc(real, training=True)
            fake_pred = disc(fake, training=True)
            d_loss = bce(tf.ones_like(real_pred), real_pred) + \
                     bce(tf.zeros_like(fake_pred), fake_pred)
        grads = tape.gradient(d_loss, disc.trainable_variables)
        d_opt.apply_gradients(zip(grads, disc.trainable_variables))

        # --- Train generator ---
        noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
        with tf.GradientTape() as tape:
            fake = gen(noise, training=True)
            preds = disc(fake, training=False)
            g_loss = bce(tf.ones_like(preds), preds)
        grads = tape.gradient(g_loss, gen.trainable_variables)
        g_opt.apply_gradients(zip(grads, gen.trainable_variables))

        if step % 50 == 0:
            print(f"Step {step}: D={d_loss:.3f}  G={g_loss:.3f}")
        if step % SAVE_EVERY == 0:
            save_images(gen, step)
        if step >= STEPS:
            break

    gen.save(os.path.join(OUT_DIR, "generator_final.h5"))
    print("✅ Done.")

if __name__ == "__main__":
    train()


🐱 Found 500 images.
Step 50: D=1.811  G=0.286
Step 100: D=0.920  G=0.733
💾 Saved ./gan_tiny_out\cats_0100.png
Step 150: D=0.712  G=0.860
Step 200: D=0.699  G=0.876
💾 Saved ./gan_tiny_out\cats_0200.png
Step 250: D=0.576  G=1.067
Step 300: D=0.391  G=1.487
💾 Saved ./gan_tiny_out\cats_0300.png
Step 350: D=0.437  G=1.394
Step 400: D=0.326  G=1.548
💾 Saved ./gan_tiny_out\cats_0400.png
Step 450: D=0.261  G=1.936
Step 500: D=0.299  G=1.981
💾 Saved ./gan_tiny_out\cats_0500.png
Step 550: D=0.316  G=2.021
Step 600: D=0.261  G=2.505
💾 Saved ./gan_tiny_out\cats_0600.png
Step 650: D=0.179  G=2.574
Step 700: D=0.152  G=2.516
💾 Saved ./gan_tiny_out\cats_0700.png
Step 750: D=0.097  G=2.635
Step 800: D=0.137  G=2.282
💾 Saved ./gan_tiny_out\cats_0800.png
Step 850: D=0.133  G=2.439
Step 900: D=0.126  G=2.520
💾 Saved ./gan_tiny_out\cats_0900.png
Step 950: D=0.112  G=2.378
Step 1000: D=0.199  G=2.463
💾 Saved ./gan_tiny_out\cats_1000.png
Step 1050: D=0.118  G=2.283
Step 1100: D=0.144  G=2.457
💾 Saved ./gan_

Step 8800: D=0.104  G=3.313
💾 Saved ./gan_tiny_out\cats_8800.png
Step 8850: D=0.202  G=3.551
Step 8900: D=0.087  G=4.029
💾 Saved ./gan_tiny_out\cats_8900.png
Step 8950: D=0.150  G=3.266
Step 9000: D=0.115  G=3.203
💾 Saved ./gan_tiny_out\cats_9000.png
Step 9050: D=0.188  G=4.107
Step 9100: D=0.102  G=3.904
💾 Saved ./gan_tiny_out\cats_9100.png
Step 9150: D=0.128  G=4.068
Step 9200: D=0.131  G=3.253
💾 Saved ./gan_tiny_out\cats_9200.png
Step 9250: D=0.171  G=3.882
Step 9300: D=0.084  G=4.391
💾 Saved ./gan_tiny_out\cats_9300.png
Step 9350: D=0.106  G=3.945
Step 9400: D=0.091  G=3.228
💾 Saved ./gan_tiny_out\cats_9400.png
Step 9450: D=0.355  G=4.585
Step 9500: D=0.104  G=3.200
💾 Saved ./gan_tiny_out\cats_9500.png
Step 9550: D=0.060  G=3.204
Step 9600: D=0.051  G=3.078
💾 Saved ./gan_tiny_out\cats_9600.png
Step 9650: D=0.070  G=4.484
Step 9700: D=0.262  G=3.969
💾 Saved ./gan_tiny_out\cats_9700.png
Step 9750: D=0.234  G=3.888
Step 9800: D=0.261  G=4.604
💾 Saved ./gan_tiny_out\cats_9800.png
Step 

Step 17300: D=0.251  G=3.289
💾 Saved ./gan_tiny_out\cats_17300.png
Step 17350: D=0.444  G=3.794
Step 17400: D=0.419  G=4.337
💾 Saved ./gan_tiny_out\cats_17400.png
Step 17450: D=0.308  G=4.014
Step 17500: D=0.483  G=4.150
💾 Saved ./gan_tiny_out\cats_17500.png
Step 17550: D=0.058  G=2.381
Step 17600: D=0.086  G=4.461
💾 Saved ./gan_tiny_out\cats_17600.png
Step 17650: D=0.795  G=3.191
Step 17700: D=0.288  G=3.510
💾 Saved ./gan_tiny_out\cats_17700.png
Step 17750: D=0.317  G=4.509
Step 17800: D=0.622  G=3.458
💾 Saved ./gan_tiny_out\cats_17800.png
Step 17850: D=0.285  G=3.678
Step 17900: D=0.221  G=3.544
💾 Saved ./gan_tiny_out\cats_17900.png
Step 17950: D=0.549  G=3.986
Step 18000: D=0.416  G=4.375
💾 Saved ./gan_tiny_out\cats_18000.png
Step 18050: D=0.072  G=3.093
Step 18100: D=0.253  G=3.224
💾 Saved ./gan_tiny_out\cats_18100.png
Step 18150: D=0.103  G=2.943
Step 18200: D=0.149  G=3.496
💾 Saved ./gan_tiny_out\cats_18200.png
Step 18250: D=0.374  G=3.191
Step 18300: D=0.655  G=3.376
💾 Saved ./g

Step 25750: D=0.563  G=3.881
Step 25800: D=0.355  G=2.918
💾 Saved ./gan_tiny_out\cats_25800.png
Step 25850: D=0.254  G=4.876
Step 25900: D=0.494  G=4.291
💾 Saved ./gan_tiny_out\cats_25900.png
Step 25950: D=0.405  G=2.942
Step 26000: D=0.203  G=3.460
💾 Saved ./gan_tiny_out\cats_26000.png
Step 26050: D=0.404  G=3.431
Step 26100: D=0.787  G=5.106
💾 Saved ./gan_tiny_out\cats_26100.png
Step 26150: D=0.135  G=2.357
Step 26200: D=0.134  G=3.247
💾 Saved ./gan_tiny_out\cats_26200.png
Step 26250: D=0.175  G=3.440
Step 26300: D=0.202  G=3.693
💾 Saved ./gan_tiny_out\cats_26300.png
Step 26350: D=0.158  G=3.669
Step 26400: D=0.493  G=2.899
💾 Saved ./gan_tiny_out\cats_26400.png
Step 26450: D=0.214  G=4.454
Step 26500: D=0.238  G=3.497
💾 Saved ./gan_tiny_out\cats_26500.png
Step 26550: D=0.186  G=2.287
Step 26600: D=0.560  G=3.234
💾 Saved ./gan_tiny_out\cats_26600.png
Step 26650: D=0.676  G=2.907
Step 26700: D=0.195  G=3.398
💾 Saved ./gan_tiny_out\cats_26700.png
Step 26750: D=0.251  G=3.220
Step 26800:

💾 Saved ./gan_tiny_out\cats_34200.png
Step 34250: D=0.325  G=4.515
Step 34300: D=0.592  G=3.213
💾 Saved ./gan_tiny_out\cats_34300.png
Step 34350: D=0.482  G=3.579
Step 34400: D=0.523  G=2.366
💾 Saved ./gan_tiny_out\cats_34400.png
Step 34450: D=0.758  G=4.370
Step 34500: D=0.158  G=4.310
💾 Saved ./gan_tiny_out\cats_34500.png
Step 34550: D=0.498  G=3.698
Step 34600: D=0.246  G=4.091
💾 Saved ./gan_tiny_out\cats_34600.png
Step 34650: D=0.243  G=2.719
Step 34700: D=0.200  G=3.239
💾 Saved ./gan_tiny_out\cats_34700.png
Step 34750: D=0.443  G=2.719
Step 34800: D=0.329  G=3.632
💾 Saved ./gan_tiny_out\cats_34800.png
Step 34850: D=0.595  G=3.107
Step 34900: D=0.249  G=4.017
💾 Saved ./gan_tiny_out\cats_34900.png
Step 34950: D=0.563  G=4.695
Step 35000: D=0.319  G=3.000
💾 Saved ./gan_tiny_out\cats_35000.png
Step 35050: D=0.397  G=4.268
Step 35100: D=0.121  G=4.319
💾 Saved ./gan_tiny_out\cats_35100.png
Step 35150: D=0.599  G=2.372
Step 35200: D=0.432  G=4.150
💾 Saved ./gan_tiny_out\cats_35200.png
St

...

💾 Saved ./gan_tiny_out\cats_59600.png
Step 59650: D=0.137  G=3.072
Step 59700: D=0.425  G=2.592
💾 Saved ./gan_tiny_out\cats_59700.png
Step 59750: D=0.278  G=2.983
Step 59800: D=0.247  G=3.280
💾 Saved ./gan_tiny_out\cats_59800.png
Step 59850: D=0.374  G=3.444
Step 59900: D=0.467  G=3.922
💾 Saved ./gan_tiny_out\cats_59900.png
Step 59950: D=0.769  G=2.930
Step 60000: D=0.287  G=3.732
💾 Saved ./gan_tiny_out\cats_60000.png
Step 60050: D=0.111  G=3.126
Step 60100: D=0.324  G=3.596
💾 Saved ./gan_tiny_out\cats_60100.png
Step 60150: D=0.356  G=2.453
Step 60200: D=0.399  G=3.451
💾 Saved ./gan_tiny_out\cats_60200.png
Step 60250: D=0.217  G=3.759
Step 60300: D=0.107  G=2.797
💾 Saved ./gan_tiny_out\cats_60300.png
Step 60350: D=0.366  G=4.097
Step 60400: D=0.204  G=3.316
💾 Saved ./gan_tiny_out\cats_60400.png
Step 60450: D=0.243  G=2.676
Step 60500: D=0.310  G=2.766
💾 Saved ./gan_tiny_out\cats_60500.png
Step 60550: D=0.147  G=2.520
Step 60600: D=0.217  G=3.571
💾 Saved ./gan_tiny_out\cats_60600.png
St

Step 68100: D=0.035  G=2.891
💾 Saved ./gan_tiny_out\cats_68100.png
Step 68150: D=0.740  G=2.739
Step 68200: D=0.310  G=3.573
💾 Saved ./gan_tiny_out\cats_68200.png
Step 68250: D=0.101  G=3.392
Step 68300: D=0.396  G=3.428
💾 Saved ./gan_tiny_out\cats_68300.png
Step 68350: D=0.265  G=2.369
Step 68400: D=0.373  G=3.474
💾 Saved ./gan_tiny_out\cats_68400.png
Step 68450: D=0.119  G=4.026
Step 68500: D=0.220  G=3.252
💾 Saved ./gan_tiny_out\cats_68500.png
Step 68550: D=0.664  G=2.522
Step 68600: D=0.501  G=3.871
💾 Saved ./gan_tiny_out\cats_68600.png
Step 68650: D=0.852  G=2.564
Step 68700: D=0.125  G=2.909
💾 Saved ./gan_tiny_out\cats_68700.png
Step 68750: D=0.280  G=4.460
Step 68800: D=0.532  G=3.070
💾 Saved ./gan_tiny_out\cats_68800.png
Step 68850: D=0.213  G=3.607
Step 68900: D=0.581  G=2.125
💾 Saved ./gan_tiny_out\cats_68900.png
Step 68950: D=0.444  G=2.999
Step 69000: D=0.134  G=2.848
💾 Saved ./gan_tiny_out\cats_69000.png
Step 69050: D=0.805  G=3.129
Step 69100: D=0.487  G=2.390
💾 Saved ./g

Step 76550: D=0.103  G=2.819
Step 76600: D=0.453  G=4.445
💾 Saved ./gan_tiny_out\cats_76600.png
Step 76650: D=0.345  G=3.136
Step 76700: D=0.385  G=2.603
💾 Saved ./gan_tiny_out\cats_76700.png
Step 76750: D=0.323  G=2.936
Step 76800: D=0.130  G=2.661
💾 Saved ./gan_tiny_out\cats_76800.png
Step 76850: D=0.231  G=3.076
Step 76900: D=0.509  G=2.643
💾 Saved ./gan_tiny_out\cats_76900.png
Step 76950: D=0.845  G=2.235
Step 77000: D=0.500  G=3.560
💾 Saved ./gan_tiny_out\cats_77000.png
Step 77050: D=0.238  G=2.714
Step 77100: D=0.106  G=2.634
💾 Saved ./gan_tiny_out\cats_77100.png
Step 77150: D=0.177  G=2.608
Step 77200: D=0.360  G=2.259
💾 Saved ./gan_tiny_out\cats_77200.png
Step 77250: D=1.162  G=2.902
Step 77300: D=0.311  G=2.651
💾 Saved ./gan_tiny_out\cats_77300.png
Step 77350: D=1.000  G=3.004
Step 77400: D=0.371  G=2.501
💾 Saved ./gan_tiny_out\cats_77400.png
Step 77450: D=0.398  G=1.908
Step 77500: D=0.611  G=4.122
💾 Saved ./gan_tiny_out\cats_77500.png
Step 77550: D=0.433  G=4.364
Step 77600:

💾 Saved ./gan_tiny_out\cats_85000.png
Step 85050: D=0.135  G=2.449
Step 85100: D=0.143  G=2.721
💾 Saved ./gan_tiny_out\cats_85100.png
Step 85150: D=0.263  G=3.052
Step 85200: D=0.727  G=3.022
💾 Saved ./gan_tiny_out\cats_85200.png
Step 85250: D=0.183  G=2.560
Step 85300: D=0.329  G=3.353
💾 Saved ./gan_tiny_out\cats_85300.png
Step 85350: D=0.086  G=3.801
Step 85400: D=0.415  G=2.885
💾 Saved ./gan_tiny_out\cats_85400.png
Step 85450: D=0.318  G=2.725
Step 85500: D=0.767  G=1.883
💾 Saved ./gan_tiny_out\cats_85500.png
Step 85550: D=0.643  G=2.467
Step 85600: D=0.342  G=3.134
💾 Saved ./gan_tiny_out\cats_85600.png
Step 85650: D=0.427  G=2.749
Step 85700: D=0.385  G=4.863
💾 Saved ./gan_tiny_out\cats_85700.png
Step 85750: D=0.157  G=6.144
Step 85800: D=0.209  G=4.138
💾 Saved ./gan_tiny_out\cats_85800.png
Step 85850: D=1.063  G=4.348
Step 85900: D=1.226  G=5.085
💾 Saved ./gan_tiny_out\cats_85900.png
Step 85950: D=0.344  G=5.063
Step 86000: D=0.260  G=2.813
💾 Saved ./gan_tiny_out\cats_86000.png
St

Step 93500: D=0.401  G=3.795
💾 Saved ./gan_tiny_out\cats_93500.png
Step 93550: D=0.169  G=3.508
Step 93600: D=0.443  G=3.760
💾 Saved ./gan_tiny_out\cats_93600.png
Step 93650: D=0.157  G=3.885
Step 93700: D=0.467  G=3.164
💾 Saved ./gan_tiny_out\cats_93700.png
Step 93750: D=0.272  G=2.981
Step 93800: D=0.374  G=2.700
💾 Saved ./gan_tiny_out\cats_93800.png
Step 93850: D=0.166  G=2.210
Step 93900: D=0.124  G=2.585
💾 Saved ./gan_tiny_out\cats_93900.png
Step 93950: D=0.134  G=2.434
Step 94000: D=0.217  G=2.408
💾 Saved ./gan_tiny_out\cats_94000.png
Step 94050: D=0.184  G=2.597
Step 94100: D=0.158  G=2.410
💾 Saved ./gan_tiny_out\cats_94100.png
Step 94150: D=0.241  G=2.232
Step 94200: D=0.149  G=2.161
💾 Saved ./gan_tiny_out\cats_94200.png
Step 94250: D=0.271  G=2.013
Step 94300: D=0.147  G=2.725
💾 Saved ./gan_tiny_out\cats_94300.png
Step 94350: D=0.109  G=3.201
Step 94400: D=0.198  G=2.730
💾 Saved ./gan_tiny_out\cats_94400.png
Step 94450: D=0.238  G=3.161
Step 94500: D=0.197  G=3.349
💾 Saved ./g